# 🦜🔗 LangChain agent using a db

In [ ]:
!pip install -r requirements.txt

In [ ]:
! [ ! -f northwind.db ] && bunzip2 northwind.db.bz2

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

In [ ]:
db = SQLDatabase.from_uri("sqlite:///northwind.db")
llm = OpenAI(temperature=0, verbose=True)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [ ]:
agent_executor.run("Describe the Orders table")

In [ ]:
agent_executor.run("Which Employee has the most orders?")